In [37]:
from imageai.Detection import ObjectDetection
import numpy as np
import os
import cv2

In [38]:
execution_path = os.getcwd()

In [39]:
detector = ObjectDetection()

In [113]:
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , r"mother_fuckin_image.jpg"), output_image_path=os.path.join(execution_path , "imagenew1.jpg"))

In [114]:
detections

[{'name': 'car',
  'percentage_probability': 52.85045504570007,
  'box_points': [156, 9, 174, 27]},
 {'name': 'person',
  'percentage_probability': 60.029393434524536,
  'box_points': [88, 77, 97, 105]},
 {'name': 'person',
  'percentage_probability': 58.618372678756714,
  'box_points': [75, 80, 86, 108]},
 {'name': 'person',
  'percentage_probability': 51.33609175682068,
  'box_points': [63, 84, 74, 113]},
 {'name': 'car',
  'percentage_probability': 52.01278328895569,
  'box_points': [140, 35, 165, 55]},
 {'name': 'car',
  'percentage_probability': 81.60916566848755,
  'box_points': [222, 36, 245, 56]},
 {'name': 'car',
  'percentage_probability': 60.60898303985596,
  'box_points': [141, 52, 171, 90]},
 {'name': 'car',
  'percentage_probability': 64.5590603351593,
  'box_points': [94, 116, 150, 171]},
 {'name': 'truck',
  'percentage_probability': 69.25075054168701,
  'box_points': [31, 136, 132, 237]},
 {'name': 'bus',
  'percentage_probability': 59.40718650817871,
  'box_points': [

In [24]:
objectList = {'car':[], 'truck':[]}
objectTypes = ['car', 'truck']
for eachObject in detections:
    name = eachObject['name']
    if name in objectTypes:
        objectList[name].append([int((eachObject['box_points'][2:][0] + eachObject['box_points'][:2][0])/2), int((eachObject['box_points'][2:][1] + eachObject['box_points'][:2][1])/2)])
#         objectList.append([eachObject['name'],[int((eachObject['box_points'][2:][0] + eachObject['box_points'][:2][0])/2), int((eachObject['box_points'][2:][1] + eachObject['box_points'][:2][1])/2)]])

In [9]:
quad_coords = {
    "lonlat": np.array([
        [51.538444, -0.102410], # corner of back telephone box
        [51.538615, -0.102251], # far bus stop marking
        [51.538229, -0.102508], # Post in front left
        [51.538191, -0.102353] # Corner of dashed line bottom right
    ]),
    "pixel": np.array([
        [131, 80],
        [213, 32],
        [25, 217],
        [342, 281]
    ])
}

In [27]:
class PixelMapper(object):
    """
    Create an object for converting pixels to geographic coordinates,
    using four points with known locations which form a quadrilteral in both planes
    Parameters
    ----------
    pixel_array : (4,2) shape numpy array
        The (x,y) pixel coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    lonlat_array : (4,2) shape numpy array
        The (lon, lat) coordinates corresponding to the top left, top right, bottom right, bottom left
        pixels of the known region
    """
    def __init__(self, pixel_array, lonlat_array):
        assert pixel_array.shape==(4,2), "Need (4,2) input array"
        assert lonlat_array.shape==(4,2), "Need (4,2) input array"
        self.M = cv2.getPerspectiveTransform(np.float32(pixel_array),np.float32(lonlat_array))
        self.invM = cv2.getPerspectiveTransform(np.float32(lonlat_array),np.float32(pixel_array))
        
    def pixel_to_lonlat(self, pixel):
        """
        Convert a set of pixel coordinates to lon-lat coordinates
        Parameters
        ----------
        pixel : (N,2) numpy array or (x,y) tuple
            The (x,y) pixel coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (lon, lat) coordinates
        """
        if type(pixel) != np.ndarray:
            pixel = np.array(pixel).reshape(1,2)
        assert pixel.shape[1]==2, "Need (N,2) input array" 
        pixel = np.concatenate([pixel, np.ones((pixel.shape[0],1))], axis=1)
        lonlat = np.dot(self.M,pixel.T)
        
        return (lonlat[:2,:]/lonlat[2,:]).T
    
    def lonlat_to_pixel(self, lonlat):
        """
        Convert a set of lon-lat coordinates to pixel coordinates
        Parameters
        ----------
        lonlat : (N,2) numpy array or (x,y) tuple
            The (lon,lat) coordinates to be converted
        Returns
        -------
        (N,2) numpy array
            The corresponding (x, y) pixel coordinates
        """
        if type(lonlat) != np.ndarray:
            lonlat = np.array(lonlat).reshape(1,2)
        assert lonlat.shape[1]==2, "Need (N,2) input array" 
        lonlat = np.concatenate([lonlat, np.ones((lonlat.shape[0],1))], axis=1)
        pixel = np.dot(self.invM,lonlat.T)
        
        return (pixel[:2,:]/pixel[2,:]).T


In [11]:
pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])

In [12]:
uv_0 = (131, 80) # Top left give way sign in frame
lonlat_0 = pm.pixel_to_lonlat(uv_0)
uv_2 = (213, 32)
lonlat_2 = pm.pixel_to_lonlat(uv_2)
uv_3 = (100,120)
lonlat_3 = pm.pixel_to_lonlat(uv_3)
uv_4 = (159,195)
lonlat_4 = pm.pixel_to_lonlat(uv_4)
uv_5 = (300, 110)
lonlat_5 = pm.pixel_to_lonlat(uv_5)

lonlat_1 = (6.603361, 52.036639) # Center of the roundabout on googlemaps
uv_1 = pm.lonlat_to_pixel(lonlat_1)

In [13]:
lonlat_4

array([[51.53824717, -0.10241868]])

In [112]:
detections

[{'name': 'apple',
  'percentage_probability': 91.35767221450806,
  'box_points': [1166, 882, 2329, 2041]}]

In [111]:
objectList

{'car': [[165, 18], [152, 45], [233, 46], [156, 71], [122, 143]],
 'truck': [[81, 186]]}

In [ ]:
51.538348, -0.102444

In [88]:
def pixelToCoor(dic):
    pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])
    coordic = {}
    for k,v in dic.items():
        coords = []
        for pix in v:
            coords.append(pm.pixel_to_lonlat(pix).tolist()[0])
        coordic[k] = coords
    return coordic

In [120]:
convcoords = pixelToCoor(objectList)

In [121]:
convcoords

{'car': [[51.53878106367503, -0.10229043089634901],
  [51.5385863351056, -0.10235664306456346],
  [51.53853099533867, -0.1022559291367989],
  [51.53846687287516, -0.10237498442851745],
  [51.538308412855656, -0.10243596755262753]],
 'truck': [[51.53825767058105, -0.1024707151003526]]}

## Old
### 51.538727, -0.102328
### 51.53834125, -0.10228305

## New Bad

### 51.538203, -0.102467
### 51.538729, -0.102364

## New New
### 51.538200, -0.102517
### 51.538732, -0.102331

In [109]:
def objselect(dic, crdx, crdy):
    for k,v in dic.items():
        for i, cd in enumerate(v):
            if cd[0] > max(crdx) or cd[0] < min(crdx) or cd[1] > max(crdy) or cd[1] < min(crdy):
                v.pop(i)
    return dic

In [122]:
objselect(convcoords, [51.538200, 51.538732], [-0.102517,  -0.102331])

{'car': [[51.5385863351056, -0.10235664306456346],
  [51.53846687287516, -0.10237498442851745],
  [51.538308412855656, -0.10243596755262753]],
 'truck': [[51.53825767058105, -0.1024707151003526]]}

In [83]:
convcoords['truck']

[[51.53825767058105, -0.1024707151003526]]

In [104]:
convcoords['car']

[[51.53878106367503, -0.10229043089634901],
 [51.5385863351056, -0.10235664306456346],
 [51.53853099533867, -0.1022559291367989],
 [51.53846687287516, -0.10237498442851745],
 [51.538308412855656, -0.10243596755262753]]